In [2]:
'''
Ideally persist work the sameway like cache.
In case of persist we can change the default persist level
How much in memory and how much on disk 

Persist takes 5 args
Whether u want to store it on:
1. Disk
2. Memory
3. Off heap 
4. Deserialized or Not
5. No of Replicas of Cache
'''

'\nIdeally persist work the sameway like cache.\nIn case of persist we can change the default persist level\nHow much in memory and how much on disk \n'

In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config('spark.sql.warehouse.dir',f'/user/{username}/warehouse'). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
orders_schema = 'order_id long, order_date date, customer_id long, order_status string'
orders_df = spark.read. \
format('csv'). \
schema(orders_schema). \
load('/public/trendytech/orders/orders_1gb.csv')

In [3]:
from pyspark.storagelevel import StorageLevel

In [4]:
#Here the arguments basically say Whether you want to:
#1. Store it to disk : YES
#2. Store it to memory: YES
#3. Off heap : NO
#4. Want it deserialized: NO, Keep it Serialized
#5. No of Replicas: 1
orders_df.persist(StorageLevel(True,False,False,False,1))

order_id,order_date,customer_id,order_status
1,2013-07-25,11599,CLOSED
2,2013-07-25,256,PENDING_PAYMENT
3,2013-07-25,12111,COMPLETE
4,2013-07-25,8827,CLOSED
5,2013-07-25,11318,COMPLETE
6,2013-07-25,7130,COMPLETE
7,2013-07-25,4530,COMPLETE
8,2013-07-25,2911,PROCESSING
9,2013-07-25,5657,PENDING_PAYMENT
10,2013-07-25,5648,PENDING_PAYMENT


In [9]:
# Lets' invoke a count on above so data is stored completely onto memory
orders_df.count()

25831125

In [7]:
orders_df.unpersist()

order_id,order_date,customer_id,order_status
1,2013-07-25,11599,CLOSED
2,2013-07-25,256,PENDING_PAYMENT
3,2013-07-25,12111,COMPLETE
4,2013-07-25,8827,CLOSED
5,2013-07-25,11318,COMPLETE
6,2013-07-25,7130,COMPLETE
7,2013-07-25,4530,COMPLETE
8,2013-07-25,2911,PROCESSING
9,2013-07-25,5657,PENDING_PAYMENT
10,2013-07-25,5648,PENDING_PAYMENT


## Serialized = Slow But Less Space
## DeSerialized = Faster But More Space
# =============================================


### Say if you execute the below query
### orders_df.persist(StorageLevel(True,True,False,False,1))

### the 2nd True arguement says keep my data on disk as well as memory and if
### it cannot fit in memory then it will go to Disk.
### First Preference is MEMORY and Second Preference is DISK.
### The 4th Argument False means keep serialized.

In [4]:
orders_df.persist(StorageLevel(True,True,False,False,2))

order_id,order_date,customer_id,order_status
1,2013-07-25,11599,CLOSED
2,2013-07-25,256,PENDING_PAYMENT
3,2013-07-25,12111,COMPLETE
4,2013-07-25,8827,CLOSED
5,2013-07-25,11318,COMPLETE
6,2013-07-25,7130,COMPLETE
7,2013-07-25,4530,COMPLETE
8,2013-07-25,2911,PROCESSING
9,2013-07-25,5657,PENDING_PAYMENT
10,2013-07-25,5648,PENDING_PAYMENT


In [6]:
orders_df.count()

25831125

### Say if you execute below query!
### orders_df.persist(StorageLevel(True,True,False,True,2))
### The 4th arguement True means it will keep it Deserialized
### That means if it doesn't fit into memory it will goto Disk.

In [8]:
orders_df.persist(StorageLevel(True,True,False,True,1))

order_id,order_date,customer_id,order_status
1,2013-07-25,11599,CLOSED
2,2013-07-25,256,PENDING_PAYMENT
3,2013-07-25,12111,COMPLETE
4,2013-07-25,8827,CLOSED
5,2013-07-25,11318,COMPLETE
6,2013-07-25,7130,COMPLETE
7,2013-07-25,4530,COMPLETE
8,2013-07-25,2911,PROCESSING
9,2013-07-25,5657,PENDING_PAYMENT
10,2013-07-25,5648,PENDING_PAYMENT


In [10]:
orders_df.count()

25831125